To run this notebook SentenceTransformer needed to be installed

In [1]:
#!pip install SentenceTransformer

## Import Libraries

In [2]:
'''basics'''
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('../..', 'src')))
sys.setrecursionlimit(20500)
import vectorize_embed as em
import pandas as pd
#import pickle5 as pickle
import pickle
import numpy as np

'''Plotting'''
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

'''features'''
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import label_binarize

'''Classifiers'''
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB


'''Metrics/Evaluation'''
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from scipy import interp
from itertools import cycle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn_hierarchical_classification.classifier import HierarchicalClassifier
from sklearn_hierarchical_classification.constants import ROOT
from sklearn_hierarchical_classification.metrics import h_fbeta_score, multi_labeled
from sklearn.pipeline import Pipeline


import warnings
warnings.filterwarnings('ignore')

import joblib

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


## Import data

In [3]:
df = pd.read_csv('../../data/processed/encoded_labels/technical_area_2.csv')
df.columns

Index(['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'technical_area_2',
       '_agrobiodiversity', '_biodiversity_financing',
       '_ecosystem_based_adaptation', '_ecosystem_based_mitigation',
       '_small_island_dev_states_program_sids',
       'chemicals_and_waste_programme', 'climate_change_adaptation_programme',
       'climate_forest_of_climate_change_mitigation',
       'climate_information_and_early_warning_systems_ci_ews',
       'climate_resilient_integrated_water_resource_coastal_area_management',
       'climate_strategies_and_policy_programme', 'cooling_alternatives',
       'cross_cutting_governance_and_learning',
       'cross_sectoral_climate_resilient_livelihoods',
       'ecosystem_based_adaptation', 'ecosystem_management_and_restoration',
       'energy_access', 'energy_efficiency',
       'energy_program_of_climate_change_mitigation',
       'environmental_governance_programme',
       'fostering_food_security_and_resilient_agricultural_systems',
       

# Compare different embeddings performances

In [4]:
categories = ['_small_island_dev_states_program_sids',
       'chemicals_and_waste_programme', 'climate_change_adaptation_programme',
       'climate_forest_of_climate_change_mitigation',
       'climate_information_and_early_warning_systems_ci_ews',
       'climate_resilient_integrated_water_resource_coastal_area_management',
       'climate_strategies_and_policy_programme', 'cooling_alternatives',
       'cross_cutting_governance_and_learning',
       'cross_sectoral_climate_resilient_livelihoods',
       'ecosystem_based_adaptation', 'ecosystem_management_and_restoration',
       'energy_access', 'energy_efficiency',
       'energy_program_of_climate_change_mitigation',
       'environmental_governance_programme',
       'fostering_food_security_and_resilient_agricultural_systems',
       'heavy_metals', 'mainstreaming_biodiversity',
       'mainstreaming_integrated_policy_and_planning_for_climate_resilient_sustainable_development',
       'miscellaneous_energy_program', 'no tag',
       'persistent_organic_pollutants',
       'protection_of_transboundary_surface_groundwater_resources_in_a_changing_climate',
       'renewable_energy', 'strengthening_conservation_areas',
       'sustainable_management_of_oceans_in_a_changing_climate',
       'waste_management']

#Turning the labels into numbers
y = pd.DataFrame(df, columns = categories)

X = df['all_text_clean'].astype('str').tolist()


## Train multiple Embeddings with SGD / OneVsRest multi-label strategy

In [5]:
#Creating a dict of the embeddings
embedding_dict = {'Glove' : 'average_word_embeddings_glove.6B.300d', 
                  'Distilbert':'distilbert-base-nli-mean-tokens', 
                  'Roberta' : 'roberta-base-nli-stsb-mean-tokens', 
                  'Bert' : 'bert-base-nli-stsb-mean-tokens'}
              
sgd_classifier = SGDClassifier(alpha=1e-06,
                               loss='log',
                               max_iter=1000,
                               penalty='l1',
                               random_state = 3,
                               tol=0.001)
model = OneVsRestClassifier(sgd_classifier)

#Train test split with stratified sampling for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .3, 
                                                    shuffle = True,  
                                                    random_state = 3)
y_train = y_train.dropna(axis=1)

#Function to get the scores for each model in a df
def model_score_df(embedding_dict, X_train, X_test, y_train, y_test, category):   
    embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in embedding_dict.items():   
        embedding_name.append(k)   
        model.fit(em.get_embeddings(v, X_train), y_train)
        
        
        # save the model to disk
        filename = '../'+category+'_'+k+'model.sav'
        joblib.dump(model, filename)
        
        y_pred = model.predict(em.get_embeddings(v, X_test))
        ac_score_list.append(accuracy_score(y_test, y_pred))
        p_score_list.append(precision_score(y_test, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['embedding_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df

      
lis = []
for category in categories:
    dic = {}
    dff = model_score_df(embedding_dict, X_train, X_test, y_train[category], y_test[category], category)
    # Using DataFrame.insert() to add a column
    dic['Category'] = category
    dic['Classifiers'] = '    \n '.join(dff.embedding_name.apply(str).tolist())
    dic['accuracy_score'] = '    \n '.join(dff.accuracy_score.apply(str).tolist()) 
    dic['precision_score'] = '   \n '.join(dff.precision_score.apply(str).tolist())
    dic['recall_score'] = '    \n '.join(dff.recall_score.apply(str).tolist())
    dic['f1_score'] = '    \n '.join(dff.f1_score.apply(str).tolist())
    lis.append(dic)
    

#print (tabulate(rows, header, tablefmt='html'))


In [6]:
from tabulate import tabulate  
header = lis[0].keys()
rows =  [x.values() for x in lis]

<table>
<thead>
<tr><th>Category                                                                                  </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>_small_island_dev_states_program_sids                                                     </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.994475138121547    
 0.994475138121547    
 0.994475138121547    
 0.994475138121547                 </td><td>0.4972375690607735   
 0.4972375690607735   
 0.4972375690607735   
 0.4972375690607735                  </td><td>0.5    
 0.5    
 0.5    
 0.5               </td><td>0.4986149584487534    
 0.4986149584487534    
 0.4986149584487534    
 0.4986149584487534           </td></tr>
<tr><td>chemicals_and_waste_programme                                                             </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0

### copy-past the output of the last cell here


In [8]:
print(tabulate(rows, header, tablefmt='html'))

<table>
<thead>
<tr><th>Category                                                                                  </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>_small_island_dev_states_program_sids                                                     </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.994475138121547    
 0.994475138121547    
 0.994475138121547    
 0.994475138121547                 </td><td>0.4972375690607735   
 0.4972375690607735   
 0.4972375690607735   
 0.4972375690607735                  </td><td>0.5    
 0.5    
 0.5    
 0.5               </td><td>0.4986149584487534    
 0.4986149584487534    
 0.4986149584487534    
 0.4986149584487534           </td></tr>
<tr><td>chemicals_and_waste_programme                                                             </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0

In [ ]:
<table>
<thead>
<tr><th>Category                                                                                  </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>_small_island_dev_states_program_sids                                                     </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.994475138121547    
 0.994475138121547    
 0.994475138121547    
 0.994475138121547                 </td><td>0.4972375690607735   
 0.4972375690607735   
 0.4972375690607735   
 0.4972375690607735                  </td><td>0.5    
 0.5    
 0.5    
 0.5               </td><td>0.4986149584487534    
 0.4986149584487534    
 0.4986149584487534    
 0.4986149584487534           </td></tr>
<tr><td>chemicals_and_waste_programme                                                             </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0    
 1.0    
 1.0    
 1.0                 </td><td>1.0   
 1.0   
 1.0   
 1.0                  </td><td>1.0    
 1.0    
 1.0    
 1.0               </td><td>1.0    
 1.0    
 1.0    
 1.0           </td></tr>
<tr><td>climate_change_adaptation_programme                                                       </td><td>Distilbert    
 Bert    
 Roberta    
 Glove              </td><td>0.9834254143646409    
 0.9834254143646409    
 0.9779005524861878    
 0.9613259668508287                 </td><td>0.9915730337078652   
 0.9915730337078652   
 0.8665254237288136   
 0.7243315508021391                  </td><td>0.75    
 0.75    
 0.7471428571428571    
 0.8995238095238096               </td><td>0.8290840415486307    
 0.8290840415486307    
 0.7943181818181819    
 0.7839727195225916           </td></tr>
<tr><td>climate_forest_of_climate_change_mitigation                                               </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.994475138121547    
 0.994475138121547    
 0.994475138121547    
 0.988950276243094                 </td><td>0.4972375690607735   
 0.4972375690607735   
 0.4972375690607735   
 0.49722222222222223                  </td><td>0.5    
 0.5    
 0.5    
 0.49722222222222223               </td><td>0.4986149584487534    
 0.4986149584487534    
 0.4986149584487534    
 0.49722222222222223           </td></tr>
<tr><td>climate_information_and_early_warning_systems_ci_ews                                      </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.9502762430939227    
 0.9502762430939227    
 0.9502762430939227    
 0.9502762430939227                 </td><td>0.47513812154696133   
 0.47513812154696133   
 0.47513812154696133   
 0.47513812154696133                  </td><td>0.5    
 0.5    
 0.5    
 0.5               </td><td>0.48725212464589235    
 0.48725212464589235    
 0.48725212464589235    
 0.48725212464589235           </td></tr>
<tr><td>climate_resilient_integrated_water_resource_coastal_area_management                       </td><td>Roberta    
 Bert    
 Glove    
 Distilbert              </td><td>0.9834254143646409    
 0.9834254143646409    
 0.9779005524861878    
 0.9779005524861878                 </td><td>0.9916666666666667   
 0.9916666666666667   
 0.4889502762430939   
 0.4889502762430939                  </td><td>0.625    
 0.625    
 0.5    
 0.5               </td><td>0.6957983193277311    
 0.6957983193277311    
 0.494413407821229    
 0.494413407821229           </td></tr>
<tr><td>climate_strategies_and_policy_programme                                                   </td><td>Glove    
 Roberta    
 Distilbert    
 Bert              </td><td>0.994475138121547    
 0.9834254143646409    
 0.9779005524861878    
 0.9613259668508287                 </td><td>0.9375   
 0.910952380952381   
 0.8914772727272727   
 0.7385714285714285                  </td><td>0.9971264367816092    
 0.8542692939244663    
 0.7828407224958949    
 0.7056650246305418               </td><td>0.9652257444764649    
 0.8803173903460435    
 0.8276190476190476    
 0.7207405774741018           </td></tr>
<tr><td>cooling_alternatives                                                                      </td><td>Distilbert    
 Glove    
 Roberta    
 Bert              </td><td>0.994475138121547    
 0.988950276243094    
 0.988950276243094    
 0.988950276243094                 </td><td>0.9972222222222222   
 0.494475138121547   
 0.494475138121547   
 0.494475138121547                  </td><td>0.75    
 0.5    
 0.5    
 0.5               </td><td>0.8319405756731661    
 0.49722222222222223    
 0.49722222222222223    
 0.49722222222222223           </td></tr>
<tr><td>cross_cutting_governance_and_learning                                                     </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.9779005524861878    
 0.9779005524861878    
 0.9779005524861878    
 0.9668508287292817                 </td><td>0.4889502762430939   
 0.4889502762430939   
 0.4889502762430939   
 0.4888268156424581                  </td><td>0.5    
 0.5    
 0.5    
 0.4943502824858757               </td><td>0.494413407821229    
 0.494413407821229    
 0.494413407821229    
 0.49157303370786515           </td></tr>
<tr><td>cross_sectoral_climate_resilient_livelihoods                                              </td><td>Glove    
 Roberta    
 Bert    
 Distilbert              </td><td>0.9558011049723757    
 0.9779005524861878    
 0.9668508287292817    
 0.9392265193370166                 </td><td>0.5656814449917899   
 0.49166666666666664   
 0.49157303370786515   
 0.4913294797687861                  </td><td>0.649812734082397    
 0.49719101123595505    
 0.49157303370786515    
 0.47752808988764045               </td><td>0.5886363636363636    
 0.49441340782122906    
 0.49157303370786515    
 0.4843304843304843           </td></tr>
<tr><td>ecosystem_based_adaptation                                                                </td><td>Bert    
 Distilbert    
 Glove    
 Roberta              </td><td>0.988950276243094    
 0.9668508287292817    
 0.994475138121547    
 0.988950276243094                 </td><td>0.6666666666666666   
 0.5714285714285714   
 0.4972375690607735   
 0.49722222222222223                  </td><td>0.9944444444444445    
 0.9833333333333334    
 0.5    
 0.49722222222222223               </td><td>0.7472067039106145    
 0.6165254237288136    
 0.4986149584487534    
 0.49722222222222223           </td></tr>
<tr><td>ecosystem_management_and_restoration                                                      </td><td>Distilbert    
 Roberta    
 Bert    
 Glove              </td><td>0.9337016574585635    
 0.9116022099447514    
 0.9226519337016574    
 0.8066298342541437                 </td><td>0.6492248062015504   
 0.5975274725274725   
 0.5907622739018088   
 0.5556513409961685                  </td><td>0.6492248062015504    
 0.6375968992248062    
 0.5907622739018088    
 0.6876614987080103               </td><td>0.6492248062015504    
 0.6128342245989304    
 0.5907622739018088    
 0.5559060637924991           </td></tr>
<tr><td>energy_access                                                                             </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.9668508287292817    
 0.9668508287292817    
 0.9613259668508287    
 0.9558011049723757                 </td><td>0.48342541436464087   
 0.48342541436464087   
 0.48333333333333334   
 0.48324022346368717                  </td><td>0.5    
 0.5    
 0.49714285714285716    
 0.4942857142857143               </td><td>0.4915730337078652    
 0.4915730337078652    
 0.49014084507042255    
 0.4887005649717514           </td></tr>
<tr><td>energy_efficiency                                                                         </td><td>Bert    
 Distilbert    
 Roberta    
 Glove              </td><td>0.9834254143646409    
 0.9723756906077348    
 0.9723756906077348    
 0.9337016574585635                 </td><td>0.9916201117318435   
 0.7388268156424581   
 0.7388268156424581   
 0.5820855614973262                  </td><td>0.7    
 0.5971590909090909    
 0.5971590909090909    
 0.6744318181818182               </td><td>0.7814889336016098    
 0.635814889336016    
 0.635814889336016    
 0.6076589595375722           </td></tr>
<tr><td>energy_program_of_climate_change_mitigation                                               </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.994475138121547    
 0.994475138121547    
 0.994475138121547    
 0.994475138121547                 </td><td>0.9972222222222222   
 0.9972222222222222   
 0.9972222222222222   
 0.9972222222222222                  </td><td>0.75    
 0.75    
 0.75    
 0.75               </td><td>0.8319405756731661    
 0.8319405756731661    
 0.8319405756731661    
 0.8319405756731661           </td></tr>
<tr><td>environmental_governance_programme                                                        </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.994475138121547    
 0.994475138121547    
 0.994475138121547    
 0.994475138121547                 </td><td>0.4972375690607735   
 0.4972375690607735   
 0.4972375690607735   
 0.4972375690607735                  </td><td>0.5    
 0.5    
 0.5    
 0.5               </td><td>0.4986149584487534    
 0.4986149584487534    
 0.4986149584487534    
 0.4986149584487534           </td></tr>
<tr><td>fostering_food_security_and_resilient_agricultural_systems                                </td><td>Bert    
 Distilbert    
 Glove    
 Roberta              </td><td>0.9668508287292817    
 0.9613259668508287    
 0.9779005524861878    
 0.9779005524861878                 </td><td>0.6165254237288136   
 0.5914772727272727   
 0.4889502762430939   
 0.4889502762430939                  </td><td>0.6165254237288136    
 0.6137005649717514    
 0.5    
 0.5               </td><td>0.6165254237288136    
 0.6011960969468052    
 0.494413407821229    
 0.494413407821229           </td></tr>
<tr><td>heavy_metals                                                                              </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.988950276243094    
 0.9834254143646409    
 0.9834254143646409    
 0.9834254143646409                 </td><td>0.9944444444444445   
 0.49171270718232046   
 0.49171270718232046   
 0.49171270718232046                  </td><td>0.6666666666666666    
 0.5    
 0.5    
 0.5               </td><td>0.7472067039106145    
 0.49582172701949867    
 0.49582172701949867    
 0.49582172701949867           </td></tr>
<tr><td>mainstreaming_biodiversity                                                                </td><td>Glove    
 Distilbert    
 Bert    
 Roberta              </td><td>0.9060773480662984    
 0.9060773480662984    
 0.8784530386740331    
 0.9005524861878453                 </td><td>0.6346899224806202   
 0.5630681818181819   
 0.5119883040935672   
 0.4604519774011299                  </td><td>0.5891787852865698    
 0.5237382378100941    
 0.5087681779298546    
 0.4880239520958084               </td><td>0.6053610362960113    
 0.5278502378394967    
 0.5091222879684418    
 0.4738372093023256           </td></tr>
<tr><td>mainstreaming_integrated_policy_and_planning_for_climate_resilient_sustainable_development</td><td>Roberta    
 Distilbert    
 Glove    
 Bert              </td><td>0.9723756906077348    
 0.8839779005524862    
 0.9668508287292817    
 0.9558011049723757                 </td><td>0.6582397003745318   
 0.5413690476190476   
 0.4888268156424581   
 0.4887005649717514                  </td><td>0.6193502824858756    
 0.6963276836158192    
 0.4943502824858757    
 0.4887005649717514               </td><td>0.6358148893360162    
 0.5488427299703263    
 0.49157303370786515    
 0.4887005649717514           </td></tr>
<tr><td>miscellaneous_energy_program                                                              </td><td>Distilbert    
 Glove    
 Roberta    
 Bert              </td><td>0.9834254143646409    
 0.9779005524861878    
 0.9779005524861878    
 0.9723756906077348                 </td><td>0.5   
 0.5   
 0.5   
 0.5                  </td><td>0.49171270718232046    
 0.4889502762430939    
 0.4889502762430939    
 0.4861878453038674               </td><td>0.49582172701949867    
 0.494413407821229    
 0.494413407821229    
 0.4929971988795518           </td></tr>
<tr><td>no tag                                                                                    </td><td>Glove    
 Bert    
 Distilbert    
 Roberta              </td><td>0.6961325966850829    
 0.6740331491712708    
 0.6629834254143646    
 0.6464088397790055                 </td><td>0.6901693667157585   
 0.6603603603603603   
 0.6552722345405272   
 0.6377777777777778                  </td><td>0.6965372907153728    
 0.6580416032470826    
 0.6598807711821411    
 0.6415525114155252               </td><td>0.6906944228677955    
 0.6590030973592618    
 0.6558828112825308    
 0.6383616383616384           </td></tr>
<tr><td>persistent_organic_pollutants                                                             </td><td>Glove    
 Roberta    
 Distilbert    
 Bert              </td><td>0.994475138121547    
 0.988950276243094    
 0.9834254143646409    
 0.9834254143646409                 </td><td>0.4972375690607735   
 0.49722222222222223   
 0.4972067039106145   
 0.4972067039106145                  </td><td>0.5    
 0.49722222222222223    
 0.49444444444444446    
 0.49444444444444446               </td><td>0.4986149584487534    
 0.49722222222222223    
 0.4958217270194986    
 0.4958217270194986           </td></tr>
<tr><td>protection_of_transboundary_surface_groundwater_resources_in_a_changing_climate           </td><td>Distilbert    
 Bert    
 Glove    
 Roberta              </td><td>1.0    
 1.0    
 0.994475138121547    
 0.994475138121547                 </td><td>1.0   
 1.0   
 0.5   
 0.5                  </td><td>1.0    
 1.0    
 0.4972375690607735    
 0.4972375690607735               </td><td>1.0    
 1.0    
 0.4986149584487534    
 0.4986149584487534           </td></tr>
<tr><td>renewable_energy                                                                          </td><td>Bert    
 Glove    
 Distilbert    
 Roberta              </td><td>0.9558011049723757    
 0.9116022099447514    
 0.9723756906077348    
 0.9613259668508287                 </td><td>0.5886363636363636   
 0.5576305220883534   
 0.4861878453038674   
 0.4860335195530726                  </td><td>0.5886363636363636    
 0.6630681818181818    
 0.5    
 0.4943181818181818               </td><td>0.5886363636363636    
 0.5766081871345029    
 0.4929971988795518    
 0.49014084507042255           </td></tr>
<tr><td>strengthening_conservation_areas                                                          </td><td>Roberta    
 Glove    
 Distilbert    
 Bert              </td><td>0.9337016574585635    
 0.9005524861878453    
 0.8066298342541437    
 0.8784530386740331                 </td><td>0.48563218390804597   
 0.4851190476190476   
 0.5022021812080537   
 0.4847560975609756                  </td><td>0.48011363636363635    
 0.4630681818181818    
 0.5119318181818182    
 0.45170454545454547               </td><td>0.4828571428571429    
 0.4738372093023256    
 0.4731808731808732    
 0.46764705882352936           </td></tr>
<tr><td>sustainable_management_of_oceans_in_a_changing_climate                                    </td><td>Bert    
 Roberta    
 Glove    
 Distilbert              </td><td>0.988950276243094    
 0.9723756906077348    
 0.9779005524861878    
 0.9779005524861878                 </td><td>0.9944444444444445   
 0.6193502824858756   
 0.49166666666666664   
 0.49166666666666664                  </td><td>0.6666666666666666    
 0.6582397003745318    
 0.49719101123595505    
 0.49719101123595505               </td><td>0.7472067039106145    
 0.6358148893360162    
 0.49441340782122906    
 0.49441340782122906           </td></tr>
<tr><td>waste_management                                                                          </td><td>Distilbert    
 Bert    
 Glove    
 Roberta              </td><td>0.9613259668508287    
 0.9558011049723757    
 0.9613259668508287    
 0.9392265193370166                 </td><td>0.6886363636363637   
 0.6552380952380952   
 0.48333333333333334   
 0.48295454545454547                  </td><td>0.6580952380952381    
 0.6552380952380952    
 0.49714285714285716    
 0.4857142857142857               </td><td>0.6718466718466718    
 0.6552380952380952    
 0.49014084507042255    
 0.4843304843304843           </td></tr>
</tbody>
</table>
